In [1]:
import sys
import shutil
import re
import math
from collections import Counter
# shutil.rmtree('TF_index')
# !zip -r TF_index.zip CTF_index
# !unzip TF_index.zip
!pip install pyspark
from pyspark import SparkContext, SparkConf

# create Spark context with necessary configuration
conf = SparkConf()
conf.setMaster('local')
conf.setAppName('TF/IDF')
sc = SparkContext.getOrCreate(conf=conf)

     |████████████████████████████████| 212.3MB 83kB/s 
     |████████████████████████████████| 204kB 17.5MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=635fbe48faf2ee19b0235483b7ab9db530ebcfe562fc38d675d73b2caaa1e271
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [6]:
# read data
# Edit file name to give your input data
dataFile = '2.txt'
stopWordFile = 'stopwords-en.txt'
# create RDDS, read stopwords file
fileRDD = sc.textFile(dataFile)


In [7]:
# Remove punctutaion function

def lower_clean_str(x):
  punc='!"#$%&\'()*+,;=?@[]^_`{|}~-'
  # punc='!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~-'
  lowercased_str = x.lower()
  for ch in punc:
    lowercased_str = lowercased_str.replace(ch, ' ')
    lowercased_str = ' '.join(lowercased_str.split())
  return lowercased_str
def lower_clean_str1(x):
  # punc='!"#$%&\'()*+,;=?@[]^_`{|}~-'
  punc='!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~-'
  lowercased_str = x.lower()
  for ch in punc:
    lowercased_str = lowercased_str.replace(ch, ' ')
    lowercased_str = ' '.join(lowercased_str.split())
  return lowercased_str

#Run remove punctuation function on data
fileRDD = sc.textFile(dataFile)
fileRDD = fileRDD.map(lower_clean_str)


In [8]:
# Remove urls
def remove_urls_tags(x):
  regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
  # regex = 'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
  url_str = re.sub(regex, " ", x)
  html_regex = r"<.*?>"
  finalStr = re.sub(html_regex, " ", url_str)
  return finalStr

fileRDD = fileRDD.map(remove_urls_tags)
fileRDD = fileRDD.map(lower_clean_str1)


In [10]:
# Remove stop words function
def remove_stop_words(stri):
  with open('stopwords-en.txt','r') as stop:
    stop_w = stop.read()
    #splitting stopwords.txt by new line
    stop_list = re.split('\n', stop_w)
    for word in stop_list:
        if word in stri:
            stri = stri.replace(" "+ word + " ", " ")
    return stri
    
fileRDD1 = fileRDD.map(remove_stop_words)
fileRDD1 = fileRDD1.map(lambda line : line.strip())
# fileRDD1.take(10)


In [12]:
def split_file(line):
    spl = line.split(" ", 1)
    return (spl[0], spl[1])
dfRDD = fileRDD1.map(split_file)
# dfRDD.take(10)

In [16]:
#  Count CTF cosine normalization, construct the above RDD into:
# (word, ([(doc-1, normalized_tf), (doc-2, normalized_tf), ...]))
def tf_construct(line):
    words = [word for word in line[1].split(" ") if word not in ['']]
    counter  = Counter(words)
    mul = [(k, v*v) for k, v in counter.items()]
    li = [v[1] for v in mul]
    eucl=math.sqrt(sum(li))
    result = [(k, (str(line[0]) + "#"+ str(v/eucl))) for k, v in mul]
    return result

tf = dfRDD.flatMap(tf_construct)
tfReduce = tf.groupByKey().map(lambda x : (x[0]+"@"+"+".join(list(x[1]))))

In [14]:
####################### step 2 output. Write this RDD to CTF_index directory.
tfReduce.saveAsTextFile('CTF_index')